In [1]:
%cd PaddleScience
! export PYTHONPATH=$PWD:$PYTHONPATH
! pip -q install -r requirements.txt

/home/aistudio/PaddleScience
/usr/bin/sh: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)


/usr/bin/sh: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)


In [2]:
import ppsci
import paddle
paddle.set_default_dtype("float32")
import numpy as np
from ppsci.utils import logger

/bin/sh: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)


W1030 02:32:25.790843   420 gpu_resources.cc:119] Please NOTE: device: 0, GPU Compute Capability: 7.0, Driver API Version: 12.0, Runtime API Version: 11.6
W1030 02:32:25.797179   420 gpu_resources.cc:149] device: 0, cuDNN Version: 8.4.


In [3]:
OUTPUT_DIR = "./output"
logger.init_logger("ppsci", f"{OUTPUT_DIR}/train.log", "info")
# set random seed for reproducibility
SEED = 1234
ppsci.utils.misc.set_random_seed(SEED)

ITERS_PER_EPOCH=1
# set model
input_key=("x", "y")
output_key=("u","v","p")
model = ppsci.arch.MLP(
         input_key, output_key, 4, 50, "tanh", input_dim=len(input_key),output_dim=len(output_key),Xavier=True
     )

In [4]:
N_train = 2601
Nb_train = 400


# Load Data
Re = 40
lam = 0.5 * Re - np.sqrt(0.25 * (Re ** 2) + 4 * (np.pi ** 2))

x = np.linspace(-0.5, 1.0, 101)
y = np.linspace(-0.5, 1.5, 101)

yb1 = np.array([-0.5] * 100)
yb2 = np.array([1] * 100)
xb1 = np.array([-0.5] * 100)
xb2 = np.array([1.5] * 100)

y_train1 = np.concatenate([y[1:101], y[0:100], xb1, xb2], 0).astype('float32')
x_train1 = np.concatenate([yb1, yb2, x[0:100], x[1:101]], 0).astype('float32')

xb_train = x_train1.reshape(x_train1.shape[0], 1).astype('float32')
yb_train = y_train1.reshape(y_train1.shape[0], 1).astype('float32')
ub_train = 1 - np.exp(lam * xb_train) * np.cos(2 * np.pi * yb_train)
vb_train = lam / (2 * np.pi) * np.exp(lam * xb_train) * np.sin(2 * np.pi * yb_train)

x_train = ((np.random.rand(N_train, 1) - 1 / 3) * 3 / 2)
y_train = ((np.random.rand(N_train, 1) - 1 / 4) * 2)

In [5]:
# Test Data
np.random.seed(SEED)
x_star = ((np.random.rand(1000, 1) - 1 / 3) * 3 / 2).astype('float32')
y_star = ((np.random.rand(1000, 1) - 1 / 4) * 2).astype('float32')

u_star = 1 - np.exp(lam * x_star) * np.cos(2 * np.pi * y_star)
v_star = (lam / (2 * np.pi)) * np.exp(lam * x_star) * np.sin(2 * np.pi * y_star)
p_star = 0.5 * (1 - np.exp(2 * lam * x_star))

In [7]:
input_train = {"x": xb_train,"y":yb_train}
label_train = {"u": ub_train,"v":vb_train}

train_dataloader_cfg = {
    "dataset": {
        "name": "NamedArrayDataset",
        "input":input_train,
        "label":label_train
    },
    "batch_size": Nb_train,
    'iters_per_epoch':ITERS_PER_EPOCH,
    "sampler": {
        "name": "BatchSampler",
        "drop_last": False,
        "shuffle": False,
    },
}
input_test = {"x": x_star,"y":y_star}
label_test = {"u": u_star,"v":v_star,"p":p_star}
valida_dataloader_cfg = {
    "dataset": {
        "name": "NamedArrayDataset",
        "input":input_test,
        "label":label_test},
    'total_size':u_star.shape[0],
    "batch_size": u_star.shape[0],
    "sampler": {
        "name": "BatchSampler",
        "drop_last": False,
        "shuffle": False,
    },
}

In [8]:
#geom = ppsci.geometry.Rectangle((-0.5, -0.5), (1.5, 1.5))
#geom = ppsci.geometry.Rectangle((0, 0), (1, 1))
geom2 = ppsci.geometry.PointCloud({"x": x_train,"y":y_train}, ("x","y"))

In [9]:
## supervised constraint s.t ||u-u_0||
sup_constraint = ppsci.constraint.SupervisedConstraint(
    train_dataloader_cfg,
    ppsci.loss.MSELoss("mean"),
    name="Sup",
)

In [10]:
# set equation constarint s.t. ||F(u)||


equation = {
        "NavierStokes": ppsci.equation.NavierStokes(nu=1.0/Re, rho=1.0, dim=2, time=False),
    }

# pde_constraint = ppsci.constraint.InteriorConstraint(
#     equation["NavierStokes"].equations,
#     {"continuity": 0, "momentum_x": 0, "momentum_y": 0},
#     geom,
#     {
#         "dataset": {"name":"IterableNamedArrayDataset"},
#         "batch_size": N_train//ITERS_PER_EPOCH,
#         "iters_per_epoch": ITERS_PER_EPOCH,
#     },
#     ppsci.loss.MSELoss("mean"),
#     name="EQ",
# )

pde_constraint_2 = ppsci.constraint.InteriorConstraint(
    equation["NavierStokes"].equations,
    {"continuity": 0, "momentum_x": 0, "momentum_y": 0},
    geom2,
    {
        "dataset": {"name":"IterableNamedArrayDataset"},
        "batch_size": N_train,
        "iters_per_epoch": ITERS_PER_EPOCH,
    },
    ppsci.loss.MSELoss("mean"),
    name="EQ2",
)

In [11]:
constraint={sup_constraint.name:sup_constraint,pde_constraint_2.name:pde_constraint_2}

In [12]:
residual_validator = ppsci.validate.SupervisedValidator(
    valida_dataloader_cfg,
    ppsci.loss.L2RelLoss(),
    metric={"L2R": ppsci.metric.L2Rel()},
    name="Residual",
)
residual_validator_mse = ppsci.validate.SupervisedValidator(
    valida_dataloader_cfg,
    ppsci.loss.MSELoss("mean"),
    metric={"MSE": ppsci.metric.MSE()},
    name="Residual",
)

# Wrap validator
validator = {residual_validator.name: residual_validator}

In [ ]:
# set optimizer
#decay_epochs=[5000,10000,60000]
EPOCHS=5000
# lr = ppsci.optimizer.lr_scheduler.Piecewise(EPOCHS, ITERS_PER_EPOCH, decay_epochs, (1e-3, 1e-4,1e-5,1e-6))()
optimizer = ppsci.optimizer.Adam(1e-3)(model)

logger.init_logger("ppsci", f"{OUTPUT_DIR}/eval.log", "info")
# initialize solver
solver = ppsci.solver.Solver(
    model=model,
    constraint=constraint,
    optimizer=optimizer,
    epochs=EPOCHS,
    iters_per_epoch=ITERS_PER_EPOCH,
    eval_during_train=False,
    log_freq=5000,
    eval_freq=1000,
    seed=SEED,
    equation=equation,
    geom=geom2,
    validator=validator,
    visualizer=None,
    eval_with_no_grad=False,
    output_dir='/home/aistudio/'
)
# train model
solver.train()
solver.eval()
solver.plot_loss_history()

In [ ]:
# set optimizer

optimizer = ppsci.optimizer.Adam(1e-4)(model)

logger.init_logger("ppsci", f"{OUTPUT_DIR}/eval.log", "info")
# initialize solver
solver = ppsci.solver.Solver(
    model=model,
    constraint=constraint,
    optimizer=optimizer,
    epochs=EPOCHS,
    iters_per_epoch=ITERS_PER_EPOCH,
    eval_during_train=False,
    log_freq=2000,
    eval_freq=2000,
    seed=SEED,
    equation=equation,
    geom=geom2,
    validator=validator,
    visualizer=None,
    eval_with_no_grad=False,
    output_dir='/home/aistudio/'
)
# train model
solver.train()
solver.eval()

In [ ]:
#set optimizer
EPOCHS=50000
optimizer = ppsci.optimizer.Adam(1e-5)(model)

logger.init_logger("ppsci", f"{OUTPUT_DIR}/eval.log", "info")
# initialize solver
solver = ppsci.solver.Solver(
    model=model,
    constraint=constraint,
    optimizer=optimizer,
    epochs=EPOCHS,
    iters_per_epoch=ITERS_PER_EPOCH,
    eval_during_train=False,
    log_freq=2000,
    eval_freq=2000,
    seed=SEED,
    equation=equation,
    geom=geom2,
    validator=validator,
    visualizer=None,
    eval_with_no_grad=False,
    output_dir='/home/aistudio/'
)
# train model
solver.train()
solver.eval()

In [ ]:
#set optimizer
EPOCHS=50000
optimizer = ppsci.optimizer.Adam(1e-6)(model)

logger.init_logger("ppsci", f"{OUTPUT_DIR}/eval.log", "info")
# initialize solver
solver = ppsci.solver.Solver(
    model=model,
    constraint=constraint,
    optimizer=optimizer,
    epochs=EPOCHS,
    iters_per_epoch=ITERS_PER_EPOCH,
    eval_during_train=False,
    log_freq=2000,
    eval_freq=2000,
    seed=SEED,
    equation=equation,
    geom=geom2,
    validator=validator,
    visualizer=None,
    eval_with_no_grad=False,
    output_dir='/home/aistudio/'
)
# train model
solver.train()
solver.eval()

In [17]:
# set optimizer
EPOCHS=5000
optimizer = ppsci.optimizer.LBFGS(max_iter=50000,tolerance_change=np.finfo(float).eps,history_size=50)(model)

logger.init_logger("ppsci", f"{OUTPUT_DIR}/eval.log", "info")
# initialize solver
solver = ppsci.solver.Solver(
    model=model,
    constraint=constraint,
    optimizer=optimizer,
    epochs=EPOCHS,
    iters_per_epoch=ITERS_PER_EPOCH,
    eval_during_train=False,
    log_freq=2000,
    eval_freq=2000,
    seed=SEED,
    equation=equation,
    geom=geom2,
    validator=validator,
    visualizer=None,
    eval_with_no_grad=False,
    output_dir='/home/aistudio/'
)
# train model
solver.train()

# evaluate after finished training
solver.eval()

[2023/10/30 03:22:30] ppsci INFO: Using paddlepaddle 2.5.0 on device Place(gpu:0)


[2023/10/30 03:22:30] ppsci INFO: Set to_static=False for forward computation.


[2023/10/30 03:22:30] ppsci INFO: [Train][Epoch 1/5000][Iter: 1/1] lr: 1.00000, loss: 0.00001, Sup: 0.00000, EQ2: 0.00000, batch_cost: 0.28448s, reader_cost: 0.00387s, ips: 10548.92451 samples/s, eta: 0:23:42


[2023/10/30 03:22:30] ppsci INFO: [Train][Epoch 1/5000][Avg] loss: 0.00001, Sup: 0.00000, EQ2: 0.00000


[2023/10/30 03:22:30] ppsci MESSAGE: Finish saving checkpoint to /home/aistudio/checkpoints/latest


...

[2023/10/30 03:28:00] ppsci INFO: [Train][Epoch 5000/5000][Avg] loss: 0.00001, Sup: 0.00000, EQ2: 0.00000


[2023/10/30 03:28:00] ppsci MESSAGE: Finish saving checkpoint to /home/aistudio/checkpoints/latest


[2023/10/30 03:28:00] ppsci INFO: [Eval][Epoch 0][Iter: 1/1] loss(Residual): 39.12681, batch_cost: 0.00971s, reader_cost: 0.00803s, ips: 103012.64751samples/s, eta: 0:00:00


[2023/10/30 03:28:00] ppsci INFO: [Eval][Epoch 0][Avg] loss(Residual): 39.12681, L2R.u(Residual): 0.00056, L2R.v(Residual): 0.00399, L2R.p(Residual): 3.70690


(inf,
 PrettyOrderedDict([('L2R',
                     {'u': Tensor(shape=[], dtype=float32, place=Place(gpu:0), stop_gradient=True,
                             0.00055963),
                      'v': Tensor(shape=[], dtype=float32, place=Place(gpu:0), stop_gradient=True,
                             0.00398842),
                      'p': Tensor(shape=[], dtype=float32, place=Place(gpu:0), stop_gradient=True,
                             3.70689750)})]))